In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import dqn
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *
from wrappers_2 import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 512, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder()
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AnnaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.qvalue_head = nn.Linear(features_dim, num_outputs)
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.qvalue_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        qvalues = self.qvalue_head(features)
        return qvalues, state

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, rew):
        if rew == 0:
            rew = -0.01
        return rew
    
def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=500)
    env.update_taskset(TaskSet(preset=['C8']))
    env = PovOnlyWrapper(env)
    env = SelectAndPlace(env)
    env = Discretization(env, flat_action_space('human-level'))
    env = RewardWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.dqn import ApexTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(ApexTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             #"gamma": 0.99,
             "num_gpus": 1,
             "num_workers": 2,
             "buffer_size": 250_000,
             "learning_starts": 10_000,
             "train_batch_size": 1000,
             "target_network_update_freq": 5000,
             #"prioritized_replay_alpha": 0.5,
             #"final_prioritized_replay_beta": 1.0,
             "min_iter_time_s": 30, 
             "rollout_fragment_length": 4,
             "collect_metrics_timeout": 1800,
             
             "v_min": -10.0,
             "v_max": 100.0,
             
             "exploration_config": {
                  "initial_epsilon": 1,
                  "epsilon_timesteps": 100_000,
                  "final_epsilon": 0.05,
              },
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "APEX C8 pretrained (AnnaCNN) r: -0.01"
                  }
              },
              #"training_intensity": 50,
              "lr": 1e-5,
             
              "evaluation_num_workers": 1,
              "evaluation_interval": 5,
              "evaluation_num_episodes": 1,
              "evaluation_config": {
                  #"input": "sampler",
                  "explore": False,  
              },
        },
        loggers=[WandbLogger])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2021-10-20 08:49:14,970	INFO wandb.py:170 -- Already logged into W&B.
2021-10-20 08:49:14,983	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


Trial name,status,loc
APEX_my_env_9a6b4_00000,RUNNING,


wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: WARNING Tried to auto resume run with id ae4f4_00000 but id 9a6b4_00000 is set.
wandb: wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
(pid=162) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=162)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=162) 2021-10-20 08:49:18,485	INFO dqn.py:188 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=162) 2021-10-20 08:49:18,485	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=167) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=167)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=169) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=169)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=162) 2021-10-20 08:49:25,129	WARNING deprecation.py:39 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
(

Result for APEX_my_env_9a6b4_00000:
  agent_timesteps_total: 25000
  custom_metrics: {}
  date: 2021-10-20_08-57-36
  done: false
  episode_len_mean: 194.7741935483871
  episode_media: {}
  episode_reward_max: -0.09000000000000141
  episode_reward_mean: -1.9091935483870903
  episode_reward_min: -4.999999999999938
  episodes_this_iter: 62
  episodes_total: 62
  experiment_id: 6c20111047d54658a08015ab604d7e9f
  hostname: linar-Z390-GAMING-X
  info:
    exploration_infos:
    - cur_epsilon: 0.0
      last_timestep: 0
    - cur_epsilon: 0.4
      last_timestep: 24655
    - cur_epsilon: 0.0006553600000000003
      last_timestep: 24967
    last_target_update_ts: 6096000
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_lr: 1.0e-05
        grad_gnorm: 0.7736769914627075
        max_q: 23.064054489135742
        mean_q: 7.344659328460693
        min_q: 3.002164840698242
    learner_queue:
      size_count: 6108
      size_mean: 14.9
      size_quantiles:
      - 6.0

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
APEX_my_env_9a6b4_00000,RUNNING,192.168.3.5:162,36,21895.7,900028,2.4632,7.16,-1.88,161.72


(pid=167) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=167)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=169) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=169)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Result for APEX_my_env_9a6b4_00000:
  agent_timesteps_total: 925028
  custom_metrics: {}
  date: 2021-10-20_15-05-27
  done: false
  episode_len_mean: 158.88
  episode_media: {}
  episode_reward_max: 7.060000000000018
  episode_reward_mean: 2.8365000000000147
  episode_reward_min: -2.3099999999999743
  episodes_this_iter: 72
  episodes_total: 2489
  experiment_id: 6c20111047d54658a08015ab604d7e9f
  hostname: linar-Z390-GAMING-X
  info:
    exploration_infos:
    - cur_epsilon: 0.0
      last_timestep: 0
    - cur_epsilon: 0.4
      last_timestep: 924419
    - cur_epsilon: 0.0006553600000000003
      last_timestep: 924787
    last_target_update_ts: 414144000
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_lr: 1.0e-05
        grad_gnorm: 1.6672449111938477
        max_q: 38.29267120361328
        mean_q: 11.324572563171387
        min_q: 3.3063716888427734
    learner_queue:
      size_count: 414160
      size_mean: 14.68
      size_quantiles:
      - 5.0
  

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
APEX_my_env_9a6b4_00000,RUNNING,192.168.3.5:162,37,22557.4,925028,2.8365,7.06,-2.31,158.88


(pid=167) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=167)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=169) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=169)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
